## Data Source

https://api.covid19india.org/  
https://api.covid19india.org/csv/

Total Confirmed Cases - https://public.flourish.studio/visualisation/2917845/ <br>
Total Recovered - https://app.flourish.studio/visualisation/2917909/ <br>
Confirmed vs Recovered - https://app.flourish.studio/visualisation/2917932/ <br>
Confirmed vs Deceased - https://app.flourish.studio/visualisation/2917972/ <br>

## Import libraries

In [1]:
import re
import requests
import json
import math
import csv
import numpy as np
import pandas as pd
import folium
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## Daily cases

In [2]:
day_wise = pd.read_csv('https://api.covid19india.org/csv/latest/case_time_series.csv')

day_wise.to_csv('./data/nation_level_daily.csv', index=False)

day_wise.tail()

,Date,Daily Confirmed,Total Confirmed,Daily Recovered,Total Recovered,Daily Deceased,Total Deceased
150,28 June,19610,549200,11631,321778,384,16488
151,29 June,18339,567539,13497,335275,417,16905
152,30 June,18256,585795,12565,347840,506,17411
153,01 July,19429,605224,12064,359904,438,17849
154,02 July,21947,627171,19999,379903,378,18227


In [3]:
def timeformat(x):
    d = datetime.strptime(x, '%d %B , %Y')
    x = d.strftime('%Y-%m-%d')
    return x

In [4]:
day_wise["Date"] = day_wise["Date"].apply(lambda x: x+", 2020")

In [5]:
day_wise["Date"] = day_wise["Date"].apply(timeformat)

In [6]:
day_wise

,Date,Daily Confirmed,Total Confirmed,Daily Recovered,Total Recovered,Daily Deceased,Total Deceased
0,2020-01-30,1,1,0,0,0,0
1,2020-01-31,0,1,0,0,0,0
2,2020-02-01,0,1,0,0,0,0
3,2020-02-02,1,2,0,0,0,0
4,2020-02-03,1,3,0,0,0,0
...,...,...,...,...,...,...,...
150,2020-06-28,19610,549200,11631,321778,384,16488
151,2020-06-29,18339,567539,13497,335275,417,16905
152,2020-06-30,18256,585795,12565,347840,506,17411
153,2020-07-01,19429,605224,12064,359904,438,17849


In [7]:
no_rows = len(day_wise)-1
perspective_deaths = math.floor(day_wise["Daily Deceased"][no_rows]/day_wise["Daily Confirmed"][no_rows]*100)

In [8]:
str(perspective_deaths)+" in 100 people die due to COVID-19 in India based on the current stats."

'1 in 100 people die due to COVID-19 in India based on the current stats.'

## Latest - State wise 

In [9]:
latest = pd.read_csv('https://api.covid19india.org/csv/latest/state_wise.csv')

latest.to_csv('./data/state_level_latest.csv', index=False)
latest.drop(columns=["Migrated_Other","Delta_Confirmed","Delta_Recovered","Delta_Deaths","State_Notes"],axis=1,inplace=True)
latest.drop(latest.index[0],inplace=True)
latest.drop(latest.index[9],inplace=True)
latest.reset_index(drop=True)

,State,Confirmed,Recovered,Deaths,Active,Last_Updated_Time,State_code
0,Maharashtra,186626,101172,8178,77260,02/07/2020 22:21:26,MH
1,Tamil Nadu,98392,56021,1321,41050,02/07/2020 21:06:27,TN
2,Delhi,92175,63007,2864,26304,02/07/2020 20:25:30,DL
3,Gujarat,33999,24601,1887,7511,02/07/2020 22:41:26,GJ
4,Uttar Pradesh,24825,17221,735,6869,02/07/2020 21:20:28,UP
5,Rajasthan,18785,15043,435,3307,03/07/2020 11:53:27,RJ
6,West Bengal,19819,13037,699,6083,02/07/2020 20:25:35,WB
7,Madhya Pradesh,14106,10815,589,2702,02/07/2020 19:06:27,MP
8,Haryana,15732,11074,251,4407,03/07/2020 15:16:27,HR
9,Karnataka,18016,8336,272,9404,02/07/2020 20:16:35,KA


In [10]:
df = pd.read_html("https://distancelatlong.com/country/india#table table-striped setBorder")

In [11]:
df = df[2]

In [12]:
latest = latest.sort_values('State')

In [13]:
line = pd.DataFrame({"States": "Ladakh", "Latitude": 34.209515, "Longitude":77.615112}, index=[17])
df2 = pd.concat([df.iloc[:16], line, df.iloc[16:]]).reset_index(drop=True)
line1 = pd.DataFrame({"States": "Gujarat", "Latitude": 22.2587, "Longitude":71.1924}, index=[11])
df3 = pd.concat([df2.iloc[:10], line1, df2.iloc[10:]]).reset_index(drop=True)
line2 = pd.DataFrame({"States": "Telangana", "Latitude": 18.1124, "Longitude":79.0193}, index=[32])
df4 = pd.concat([df3.iloc[:31], line2, df3.iloc[31:]]).reset_index(drop=True)
df4.sort_values("States")
df4.reset_index(drop=True)
df = df4

In [14]:
df = df.sort_values("States")
df.drop(columns="States",axis=1,inplace=True)

In [15]:
latest = latest.reset_index(drop=True)
latest = pd.concat([latest,df],axis=1)

In [16]:
latest

,State,Confirmed,Recovered,Deaths,Active,Last_Updated_Time,State_code,Latitude,Longitude
0,Andaman and Nicobar Islands,116,51,0,65,03/07/2020 10:05:29,AN,11.667026,92.735983
1,Andhra Pradesh,16934,7632,206,9096,03/07/2020 13:37:27,AP,14.750429,78.570026
2,Arunachal Pradesh,232,71,1,160,02/07/2020 23:49:30,AR,27.100399,93.616601
3,Assam,9435,6107,14,3311,03/07/2020 01:05:28,AS,26.749981,94.216667
4,Bihar,10914,7994,78,2842,03/07/2020 13:16:29,BR,25.785414,87.479973
5,Chandigarh,450,389,6,55,02/07/2020 20:06:50,CH,30.719997,76.780006
6,Chhattisgarh,3013,2362,14,637,02/07/2020 22:21:33,CT,22.090420,82.159987
7,Dadra and Nagar Haveli and Daman and Diu,268,125,0,142,03/07/2020 15:16:30,DN,20.266578,73.016618
8,Delhi,92175,63007,2864,26304,02/07/2020 20:25:30,DL,28.669993,77.230004
9,Goa,1482,734,4,744,02/07/2020 20:47:29,GA,15.491997,73.818001


In [17]:
latest["Last_Updated_Time"] = latest["Last_Updated_Time"].apply(lambda x:x.split()[0])

In [18]:
latest = latest.astype(str)
latest.to_csv("latest.csv",index=False)

In [21]:
india = folium.Map(location = [20.5937,78.9629],zoom_start=4.5)

for date,state,lat,long,total_cases,Death,Recov,Active in zip(list(latest["Last_Updated_Time"]),list(latest['State']),list(latest['Latitude']),list(latest['Longitude']),list(latest['Confirmed']),list(latest['Deaths']),list(latest['Recovered']),list(latest['Active'])):
    folium.CircleMarker(location = [lat,long],
                       radius = 5,
                       color='red',
                       fill = True,
                       fill_color="red").add_to(india)
    folium.Marker(location = [lat,long],
                  popup=folium.Popup(('<strong><font color= blue>Last Updated: '+date+'</font></striong><br>' +
                    '<strong><b>State  : '+state+'</strong> <br>' +
                    '<strong><b>Total Cases : '+total_cases+'</striong><br>' +
                    '<strong><font color= red>Deaths : </font>'+Death+'</striong><br>' +
                    '<strong><font color=green>Recoveries : </font>'+Recov+'</striong><br>'+
                    '<strong><b>Active Cases : '+Active+'</striong>'  ),max_width=200)).add_to(india)
india

In [20]:
india.save(outfile="india.html")